In [ ]:
import json
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from shapely.geometry import Polygon
import numpy as np

from gnn_package import preprocessing
import gnn_package.src.preprocessing.graph_utils as ppgu
import gnn_package.src.preprocessing.graph_manipulation as ppgm
import gnn_package.src.preprocessing.graph_analysis as ppga
import gnn_package.src.preprocessing.graph_computation as ppgc
import gnn_package.src.preprocessing.graph_visualization as ppgv

In [ ]:
# ppgu.get_sensor_name_id_map()

In [ ]:
public_sensors_gdf = ppgu.read_or_create_public_sensors_nodes()
private_sensors_gdf = ppgu.read_or_create_private_sensor_nodes()

# Get the bounding box
bbox_transformed = ppgu.get_bbox_transformed()
public_sensors_clipped_gdf = public_sensors_gdf.clip(bbox_transformed)
private_sensors_clipped_gdf = private_sensors_gdf.clip(bbox_transformed)

# Combine the public and private sensors whilst keeping the index values of both
sensor_nodes_gdf = pd.concat(
    [public_sensors_clipped_gdf, private_sensors_clipped_gdf], axis=0
)
bbox_gdf = ppgu.get_bbox_transformed()

In [ ]:
sensor_nodes_gdf

In [ ]:
public_sensors_gdf

In [ ]:
public_sensors_clipped_gdf

In [ ]:
# Example usage
place_name = "Newcastle upon Tyne, UK"  # Replace with your area of interest

# Get the network
network_gdf = ppgu.get_street_network_gdfs(place_name)
clipped_network_gdf = gpd.clip(network_gdf, bbox_gdf)

In [ ]:
# visualize_network_components(clipped_network_gdf) # currently not working

In [ ]:
snapped_sensor_nodes_gdf = ppgm.snap_points_to_network(
    sensor_nodes_gdf, clipped_network_gdf
)

In [ ]:
clipped_network_gdf = ppgm.explode_multilinestrings(clipped_network_gdf)

connected_network_gdf = ppgm.connect_components(clipped_network_gdf)

In [ ]:
# Plot the gdfs
fig, ax = plt.subplots(figsize=(10, 10))
connected_network_gdf.plot(ax=ax, color="black", alpha=0.5)
snapped_sensor_nodes_gdf.plot(ax=ax, color="red", alpha=0.5)

In [ ]:
ppga.validate_snapped_points(snapped_sensor_nodes_gdf, connected_network_gdf)

In [ ]:
# compute shortest paths
shortest_paths_gdf = ppgc.compute_shortest_paths(
    connected_network_gdf, snapped_sensor_nodes_gdf
)

In [ ]:
# Plot the gdfs
fig, ax = plt.subplots(figsize=(10, 10))
connected_network_gdf.plot(ax=ax, color="black", alpha=0.5)
snapped_sensor_nodes_gdf.plot(ax=ax, color="red", alpha=0.5)
shortest_paths_gdf.plot(ax=ax, color="orange", alpha=0.5)

In [ ]:
G = ppgc.create_weighted_graph_from_paths(shortest_paths_gdf)

In [ ]:
ppgv.visualize_sensor_graph(G, snapped_sensor_nodes_gdf)

In [ ]:
adj_matrix_dense, node_ids = ppgu.graph_to_adjacency_matrix_and_nodes(G)

# # Get a sorted list of node IDs to ensure consistent ordering
# node_ids = sorted(list(G.nodes()))

# # Create the adjacency matrix using NetworkX's built-in function
# adj_matrix = nx.adjacency_matrix(G, nodelist=node_ids, weight="weight")

# # Convert to dense numpy array for easier viewing
# adj_matrix_dense = adj_matrix.todense()

# # Create a DataFrame for better visualization
# adj_df = pd.DataFrame(adj_matrix_dense, index=node_ids, columns=node_ids)

# print("\nFirst few rows and columns of adjacency matrix:")
# print(adj_df.iloc[:5, :5])

print("Shape of adjacency matrix:", adj_matrix_dense.shape)
# Basic statistics
non_zero_weights = adj_matrix_dense[adj_matrix_dense > 0]
print("\nDistance statistics:")
print(f"Minimum distance: {np.min(non_zero_weights):.2f} meters")
print(f"Maximum distance: {np.max(non_zero_weights):.2f} meters")
print(f"Average distance: {np.mean(non_zero_weights):.2f} meters")

In [ ]:
# Create the heatmap figure
fig = go.Figure(
    data=go.Heatmap(
        z=adj_matrix_dense,
        x=node_ids,
        y=node_ids,
        colorscale="YlOrRd",
        colorbar=dict(title="Distance (meters)", thickness=15, x=1.1),
        hoverongaps=False,
        hovertemplate="From Node: %{y}<br>"
        + "To Node: %{x}<br>"
        + "Distance: %{z:.2f} meters<br>"
        + "<extra></extra>",  # This removes the secondary box in the hover
    )
)

# Update the layout
fig.update_layout(
    title="Sensor Network Adjacency Matrix",
    width=800,
    height=600,
    xaxis=dict(
        title="To Node",
        side="bottom",
        scaleanchor="y",
        scaleratio=1,
    ),
    yaxis=dict(
        title="From Node", autorange="reversed"  # This puts (0,0) at the top-left
    ),
    margin=dict(l=60, r=100, b=60, t=40, pad=4),
)

# Show the plot
fig.show()

In [ ]:
ppgu.save_graph_data(adj_matrix_dense, node_ids, prefix="publ_priv_test")